In [1]:
from __future__ import annotations
from typing import Tuple
import numpy as np
from collections import defaultdict
from time import perf_counter
from tqdm.auto import trange

from hypercube import Hypercube

Array = np.ndarray


class Flowtree:

    def __init__(self, points: Array) -> None:
        self.original_data = points
        self.X = points + 0
        self.N, self.d = self.X.shape
        self.normalize()

    def normalize(self) -> None:
        min_dist = np.inf
        for i in range(self.N):
            for j in range(i+1, self.N):
                dist = self.X[i] - self.X[j]
                min_dist = min(min_dist, np.sum(dist**2))
        self.X /= np.sqrt(min_dist)
        self.X += np.min(self.X)
        self.phi = np.max(self.X)
    
    def __repr__(self) -> str:
        return self.X.__repr__()
        
    def construct_quadtree(self) -> None:
        # Construct H
        H0 = np.tile([[-self.phi], [self.phi]], self.d).T
        sigma = self.phi * np.random.rand(self.d, 1)
        H = H0 + np.tile(sigma, (1, 2))
        

In [2]:
d = 5
N = 20

x = Flowtree(np.random.rand(N, d))
x.construct_quadtree()

In [5]:
def normalize(X) -> None:
    N, d = X.shape
    min_dist = np.inf
    for i in trange(N):
        for j in range(i+1, N):
            dist = np.max(np.abs(X[i] - X[j]))
            min_dist = min(min_dist, dist)
    X /= np.sqrt(min_dist)
    X -= np.min(X)
    return X

def aux_build(hypercube, depth=0):
    tree = []
    if depth > 100:
        return tree
    for child in hypercube.divide():
        if isinstance(child, int):
            tree.append(child)
        elif isinstance(child, Hypercube):
            tree.append(aux_build(child, depth+1))
        else:
            raise TypeError(child)
    return tree

class Quadtree:

    def __init__(self,
        points, # k, d
    ) -> None:
        self.original_data = points
        self.d = self.original_data.shape[1]

    def normalize(self):
        self.X = normalize(self.original_data + 0)
        self.phi = np.max(self.X)

    def build(self):
        assert hasattr(self, 'phi')
        H0 = np.tile([[-self.phi], [self.phi]], self.d).T
        sigma = self.phi * np.random.rand(self.d, 1)
        H = H0 + np.tile(sigma, (1, 2))
        return aux_build(Hypercube(H, self.X, 0))

In [6]:
d = 10
N = 1000

Q = Quadtree(np.random.rand(N, d))
Q.normalize()
Q.build()

  0%|          | 0/1000 [00:00<?, ?it/s]

[[0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4, [0, 1, 2], 7, 8, 9, [0, 1], 12, 14, 15, 16, 17],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [0, 1],
 4,
 [0,
  [0, 1],
  [0, 1],
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  [0, 1],
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  21,
  23,
  25],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [0, 1, 2, 3, 4, 5, 6, 7],
 [0, 1],
 10,
 [0, 1, 2, [0, 1], 4, 5, [0, 1], 7, [0, 1, 2], 9, 10, 12, [0, 1], 14, 18, 20],
 12,
 [0, [0, 1], 3, 4, 5],
 [0, 1],
 [[0, 1], 1, 2, 3, 4, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [0, 1, 2, 3, 4],
 [0, 1],
 20,
 [0, 1],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4, 5],
 [0, [0, 1], 2, [0, 1, 2], 5, 6, 7, 8, 11, 12, 13],
 [[0, 1]],
 [0, 1, 2, 3],
 27,
 [[0, 1], 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, [0, 1], 14],
 [[0, 1], 1, 2],
 33,
 [0, 1, 2, 3, 4],
 [[0, 1],
  1,
  [0, 1],
  3,
  4,
  [0, 1, 2],
  6,
  [0, 1],
  9,
  11,
  12,
  13,
  14,
  15,
  17,
  19,
  20,
  21,
  22,
  23,